In [9]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [10]:
# Cargar los datos
df_entrenamiento = pd.read_csv('train.csv')
df_prueba = pd.read_csv('test.csv')

print("Dimensiones de los datos de entrenamiento:", df_entrenamiento.shape)
print("Dimensiones de los datos de prueba:", df_prueba.shape)

Dimensiones de los datos de entrenamiento: (9239, 10)
Dimensiones de los datos de prueba: (2309, 9)


In [11]:
# Convertir participación a valor numérico
df_entrenamiento['engagement'] = df_entrenamiento['engagement'].astype(int)

# Separar características y etiquetas
X = df_entrenamiento.drop(columns=['id', 'engagement'])
y = df_entrenamiento['engagement']

# Normalizar las características
escalador = StandardScaler()
X_escalada = escalador.fit_transform(X)

# Dividir los datos en conjunto de entrenamiento y de validación
X_entrenamiento, X_val, y_entrenamiento, y_val = train_test_split(X_escalada, y, test_size=0.2, random_state=42)

print("Dimensiones de los datos con los que se entrenará:", X_entrenamiento.shape)
print("Dimensiones de los datos de validación:", X_val.shape)

Dimensiones de los datos con los que se entrenará: (7391, 8)
Dimensiones de los datos de validación: (1848, 8)


In [313]:
# Definir los modelos
modelos = {
    # 'Regresión logística': LogisticRegression(max_iter=1000, random_state=42),
    # 'k vecinos más cercanos': KNeighborsClassifier(),
    # 'Clasificador de margen máximo': SVC(probability=True, random_state=42),
    # 'Árbol de decisión': DecisionTreeClassifier(),
    # 'Bosque aleatorio': RandomForestClassifier(random_state=42),
    # 'Potencialización de gradiente': GradientBoostingClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42),
    # 'LightGBM': LGBMClassifier(random_state=42)
}

In [362]:
# Definir la cuadrícula de hiperparámetros
cuadriculas_params = {
    # 'Regresión logística': [
    #     {
    #         'C': [0.01, 0.02, 0.05, 0.1, 1],
    #         'penalty': ['l1', 'l2'],
    #         'solver': ['liblinear', 'saga'],
    #         'class_weight': [None, 'balanced']
    #     },
    #     {
    #         'C': [0.01, 0.02, 0.05, 0.1, 1],
    #         'penalty': ['elasticnet'],
    #         'solver': ['saga'],
    #         'class_weight': [None, 'balanced'],
    #         'l1_ratio': [0.1, 0.5, 0.9]
    #     }
    # ],
    # 'k vecinos más cercanos': {
    #     'n_neighbors': [49, 51, 53, 55, 57, 59, 61],
    #     'weights': ['distance'],
    #     'p': [1, 2, 3, 4, 5],
    #     'metric': ['minkowski'],
    #     'algorithm': ['ball_tree']
    # }
    # 'Clasificador de margen máximo': [
    #     {
    #         'C': [0.002, 0.005, 0.01, 0.02, 0.05],
    #         'kernel': ['rbf'],
    #         'gamma': ['scale', 'auto', 0.002, 0.005, 0.01, 0.02, 0.05]
    #     },
    #     {
    #         'C': [0.001, 0.01, 0.1, 1.0],
    #         'kernel': ['sigmoid'],
    #         'gamma': ['scale', 'auto', 0.0001, 0.001, 0.01, 0.1, 1.0],
    #         'coef0': [0.01, 0.1, 1.0]
    #     },
    #     {
    #         'C': [1, 2, 3],
    #         'kernel': ['poly'],
    #         'gamma': [0.04, 0.05, 0.1],
    #         'degree': [3],
    #         'coef0': [10.0, 13.0, 19.0]
    #     }
    # ]
    # 'Árbol de decisión': {
    #     'max_depth': [10, None],
    #     'min_samples_split': [5, 6],
    #     'min_samples_leaf': [2, 4, 5],
    #     'max_features': [0.5, 0.8, None],
    #     'criterion': ['gini', 'entropy', 'log_loss'],
    #     'splitter': ['best', 'random'],
    #     'class_weight': ['balanced', None],
    #     'max_leaf_nodes': [30, 40, None],
    #     'min_weight_fraction_leaf': [0.0, 0.01],
    #     'min_impurity_decrease': [0.0, 0.01, 0.02]
    # }
    # 'Bosque aleatorio': {
    #     'n_estimators': [200],
    #     'max_depth': [15],
    #     'min_samples_split': [5],
    #     'min_samples_leaf': [2],
    #     'max_features': ['log2'],
    #     'class_weight': ['balanced'],
    #     'criterion': ['gini', 'entropy']
    # },
    # 'Potencialización de gradiente': {
    #     'n_estimators': [100],
    #     'max_depth': [5],
    #     'learning_rate': [0.1],
    #     'min_samples_split': [2],
    #     'min_samples_leaf': [2],
    #     'criterion': ['friedman_mse'],
    #     'loss': ['log_loss', 'exponential'],
    #     'min_weight_fraction_leaf': [0.02, 0.03]
    # },
    'XGBoost': {
        'n_estimators': [100],
        'max_depth': [5],
        'learning_rate': [0.1],
        'min_child_weight': [5],
        'subsample': [1.0],
        'gamma': [0.1],
        'reg_lambda': [0, 1],
        'objective': ['binary:logistic', 'binary:hinge', 'binary:logitraw'],
        'monotone_constraints': [(1, -1)],
        'interaction_constraints': ['[[0, 1], [2, 3, 4]]'],
    },
    # 'LightGBM': {
    #     'n_estimators': [100, 200],
    #     'max_depth': [-1, 5, 10],
    #     'learning_rate': [0.01, 0.1, 0.2],
    #     'num_leaves': [20, 40, 60],
    #     'min_child_samples': [20, 30, 50]
    # }
}

In [363]:
# Evaluar los modelos
mejor_puntaje = -float('inf')
mejor_modelo = None
resultados = {}

for nombre, modelo in modelos.items():
    print("Modelo:", nombre)

    # Validación cruzada con el criterio de área bajo la curva característica operativa del receptor (ROC AUC)
    busqueda = GridSearchCV(estimator=modelo, param_grid=cuadriculas_params[nombre], cv=5, scoring='roc_auc', n_jobs=-1, verbose=3)

    # Entrenar el modelo
    busqueda.fit(X_entrenamiento, y_entrenamiento)

    # Obtener los mejores hiperparámetros
    mejores_params = busqueda.best_params_

    # Evaluar el modelo en el conjunto de validación
    y_val_pred = busqueda.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_val_pred)

    resultados[nombre] = {
        'mejores_params': mejores_params,
        'roc_auc': roc_auc
    }

    print(f"Mejores hiperparámetros: {mejores_params}, ROC AUC (validación) = {roc_auc:.4f}")

    # Guardar el mejor modelo
    if roc_auc > mejor_puntaje:
        mejor_puntaje = roc_auc
        mejor_modelo = busqueda.best_estimator_

print(f"\nModelo con el mejor rendimiento: {mejor_modelo}")

Modelo: XGBoost
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 2/5] END gamma=0.1, interaction_constraints=[[0, 1], [2, 3, 4]], learning_rate=0.1, max_depth=5, min_child_weight=5, monotone_constraints=(1, -1), n_estimators=100, objective=binary:logistic, reg_lambda=0, subsample=1.0;, score=0.858 total time=   0.1s
[CV 1/5] END gamma=0.1, interaction_constraints=[[0, 1], [2, 3, 4]], learning_rate=0.1, max_depth=5, min_child_weight=5, monotone_constraints=(1, -1), n_estimators=100, objective=binary:logistic, reg_lambda=0, subsample=1.0;, score=0.867 total time=   0.1s
[CV 4/5] END gamma=0.1, interaction_constraints=[[0, 1], [2, 3, 4]], learning_rate=0.1, max_depth=5, min_child_weight=5, monotone_constraints=(1, -1), n_estimators=100, objective=binary:logistic, reg_lambda=0, subsample=1.0;, score=0.901 total time=   0.1s
[CV 5/5] END gamma=0.1, interaction_constraints=[[0, 1], [2, 3, 4]], learning_rate=0.1, max_depth=5, min_child_weight=5, monotone_constraints=(1, -1), n_

In [21]:
# Normalizar el conjunto de prueba
X_prueba_escalada = escalador.transform(df_prueba.drop(columns=['id']))

# Predecir con el mejor modelo encontrado
mejor_modelo.fit(X_entrenamiento, y_entrenamiento)
y_prueba_pred_proba = busqueda.predict_proba(X_prueba_escalada)[:, 1]

# Guardar los resultados en un archivo
resultados = pd.DataFrame({'id': df_prueba['id'], 'engagement': y_prueba_pred})
resultados.to_csv('resultados.csv', index=False)